In [1]:
import re
import os

In [2]:
def latex_tree_table(f_name):

    with open(f_name,"r") as f:
        data = f.read()

    data = re.sub(r"[\n]+","\n",data)
    data = data[:data.index("number of visited networks")]
    data = data.strip("\n")
    data = [ln.split() for ln in data.split("\n")]
    data = \
    [[wd.replace(ln[0],"") if wd.replace("!","") == ln[0] and i!=0 else wd for i,wd in enumerate(ln)] for ln in data]
    data = [[wd.replace("032_","").replace("_001","").replace("_",",") for wd in ln] for ln in data]
    data = [[re.sub(r"^0","", wd).replace(",0",",").replace("!0","!") for wd in ln] for ln in data]
    data = [[re.sub(r"[!]+",r"\\bf ",wd) for wd in ln] for ln in data]
    data = "\\\\\n".join([" & ".join(ln) for ln in data])
    data = data + "\\\\"
    data = data.replace("16b","16").replace("8b","8").replace("32b","32")
    
    print(data)

In [3]:
def latex_tree_tables():

    base_path = "/home/lucas/Documents/perceptronac/complexity/scripts/glch_experiments/glch_results/"
    for method in ["gift_wrapping","tie_break","angle_rule"]:
        for title in ["rate_vs_params","rate_vs_energy_noisy","model_bits_vs_data_bits"]:
            for constraint in ["unconstrained","constrained"]:
                if (method != "tie_break") or (constraint != "unconstrained"):
                    latex_tree_table(os.path.join(base_path,f"glch2D_{method}_{constraint}_{title}_tree.txt"))
                    print("\n")


In [5]:
# latex_tree_tables()